##### Calling logging configuration file

In [0]:
%run ./logging_config

##### Calling configs file

In [0]:
%run ./configs

#####calling functions notebook

In [0]:
%run ./functions

In [0]:
log = get_logger("main")
log.info("Logger initialised")

In [0]:
from __future__ import annotations

log = get_logger("main")

# Example: allow runtime overrides via widgets (optional)
# overrides = {
#     "paths": {
#         "raw": dbutils.widgets.get("raw_path"),
#         "bronze": dbutils.widgets.get("bronze_path"),
#         "silver": dbutils.widgets.get("silver_path"),
#     },
#     "source": {
#         "since_date": dbutils.widgets.get("since_date"),
#     },
# }
overrides = {}

cfg = load_config(env="dev", overrides=overrides)
log.info("Loaded configuration:\n" + show_config(cfg))

try:
    # 1) READ RAW
    raw_path = cfg["paths"]["raw"]
    raw_df = read_json_files(raw_path, mode=cfg["io"]["read_mode"])
    expect_non_empty(raw_df, step="read_raw")

    # 2) TRANSFORM
    bronze_df = flatten_neo_feed(raw_df)
    expect_non_empty(bronze_df, step="flatten")

    # 3) WRITE BRONZE
    write_delta(bronze_df, cfg["paths"]["bronze"],
                mode=cfg["io"]["write_mode"],
                merge_schema=cfg["io"]["merge_schema"])

    log.info("Pipeline completed successfully.")

except ConfigError as e:
    log.error(f"Configuration error: {e}")
    raise

except DataQualityError as e:
    log.error(f"Data quality error: {e}")
    # Decide whether to fail the job or just warn; here we fail:
    raise

except Exception as e:
    # Final guardrail
    log.exception(f"Unhandled error: {e}")
    raise
